In [444]:
import pandas as pd

In [445]:
frequentation = pd.read_csv("frequentation.csv")
menu = pd.read_csv("menus.csv")


### NETTOYAGE

## frequentation

In [446]:
frequentation2 = frequentation.copy()

ind2 = frequentation2[frequentation2.reel == 0].index
frequentation2.drop(ind2, inplace = True)



#j'enleve toute les lignes mercredi car elles concernent les centre aérées
ind = frequentation2[frequentation2.jour == "Mercredi"].index
frequentation2.drop(ind, inplace = True)

ind5 = frequentation2[frequentation2.vacances == 1].index
frequentation2.drop(ind5, inplace = True)


ind6 = frequentation2[frequentation2.greves == 1].index
frequentation2.drop(ind6, inplace = True)

# je remet les effectifs correcte car parfois dans la même années scolaire l'effectif change d'un mois à l'autre
frequentation2 = frequentation2.replace(17652, 17373)
frequentation2 = frequentation2.replace(18497, 18633)


frequentation2["ratio"] = frequentation2.reel/frequentation2.effectif*100


# on ne garde que les fréquentations supérieur ou égale à 60% car en dessous il y a des varitions trop hasardeuses
ind3 = frequentation2[frequentation2.ratio < 60 ].index
frequentation2.drop(ind3, inplace = True)


# On créé le dataframe sur lequel on va travailler
FM = frequentation2.merge(menu, how = "inner", on = "date")
FM["ratio"] = FM.reel/FM.effectif*100

## menu

In [447]:
ind7 = menu[menu.menu == "[]"].index
menu.drop(ind7, inplace = True)

ind8 = menu[menu.menu == "['repas greves']"].index
menu.drop(ind8, inplace = True)

ind9 = menu[menu.menu == "['grèves']"].index
menu.drop(ind9, inplace = True)


ind11 = menu[menu.menu == "['gréves', 'repas de depannage', 'ou froid']"].index
menu.drop(ind11, inplace = True)


ind111 = menu[menu.menu == "['menu depannage ']"].index
menu.drop(ind111, inplace = True)

In [448]:
#plus tard pour notre anova on aura besoin de chaque mois traitable individuellement 
#on créé une colonne par mois dans notre dataframe

frequentation3 = pd.get_dummies(FM.mois)
FM = FM.join(frequentation3)

In [449]:
#plus tard pour notre anova on aura besoin de chaque jpur traitable individuellement 
#on créé une colonne par jour dans notre dataframe

frequentation4 = pd.get_dummies(FM.jour)
FM = FM.join(frequentation4)

In [450]:
frequentation2 = frequentation2.drop(columns = "Unnamed: 0")
menu = menu.drop(columns = "Unnamed: 0")

### ANOVA MENU

In [451]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [452]:
lm = ols('reel ~ effectif+viande+poisson+Fevrier+Juin+Juillet+Septembre+Lundi+Vendredi+inc_grippe+veille_vacances+noel+veille_ferie -1', data = FM).fit()
import statsmodels.api; 
table = sm.stats.anova_lm(lm)
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   reel   R-squared (uncentered):                   0.999
Model:                            OLS   Adj. R-squared (uncentered):              0.999
Method:                 Least Squares   F-statistic:                          1.125e+05
Date:                Wed, 19 Jul 2023   Prob (F-statistic):                        0.00
Time:                        12:14:34   Log-Likelihood:                         -7294.0
No. Observations:                1003   AIC:                                  1.461e+04
Df Residuals:                     990   BIC:                                  1.468e+04
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
effectif            0.7292      0.002    394.604      0.000       0.726       0.733
viande           -339.2017     35.005     -9.690      0.000    -407.894    -270.509
poisson            80.6714     34.798      2.318      0.021      12.385     148.957
Fevrier          -151.2662     44.668     -3.386      0.001    -238.921     -63.611
Juin             -603.1757     36.175    -16.674      0.000    -674.164    -532.187
Juillet         -1587.3576    107.332    -14.789      0.000   -1797.982   -1376.733
Septembre        -186.9655     34.716     -5.386      0.000    -255.090    -118.841
Lundi            -433.0092     27.583    -15.698      0.000    -487.138    -378.881
Vendredi          184.7106     35.795      5.160      0.000     114.469     254.953
inc_grippe         -0.0149      0.003     -5.904      0.000      -0.020      -0.010
veille_vacances  -220.9788     69.385     -3.185      0.001    -357.137     -84.821
noel             1147.8894    133.562      8.594      0.000     885.793    1409.986
veille_ferie     -324.2608    118.175     -2.744      0.006    -556.164     -92.358
==============================================================================
Omnibus:                      100.847   Durbin-Watson:                   1.156
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              350.169
Skew:                          -0.448   Prob(JB):                     9.16e-77
Kurtosis:                       5.753   Cond. No.                     2.30e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 2.3e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [453]:
import numpy as np
from sklearn import linear_model
regr = linear_model.LinearRegression()
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Jeu d'entraînement jeu de test 

In [454]:
FM.annee_scolaire.unique()

array(['2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015',
       '2015-2016', '2016-2017', '2017-2018', '2018-2019', '2019-2020'],
      dtype=object)

In [455]:
FM_train = FM[FM['annee_scolaire'].isin(['2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015','2015-2016', '2016-2017', '2017-2018'])] 
FM_test = FM[FM['annee_scolaire'].isin(['2018-2019'])]


In [456]:
xtrain = FM_train[["effectif","viande","poisson","Fevrier","Juin","Juillet","Septembre","Lundi","Vendredi","veille_vacances","noel","veille_ferie"]]
xtest =  FM_test[["effectif","viande","poisson","Fevrier","Juin","Juillet","Septembre","Lundi","Vendredi","veille_vacances","noel","veille_ferie"]]
ytrain = FM_train[['reel']]
ytest = FM_test[['reel']]

# Regression lineaire (DEUXIEME)

In [457]:
regr.fit(xtrain,ytrain)

LinearRegression()

In [458]:
ypred_train = regr.predict(xtrain)

In [459]:
ypred_test = regr.predict(xtest)

In [460]:
# ici on créé une varible de prediction pour pouvoir jouer sur le pourcentage de repas que l'on rajoute 
# pour optimiser notre prédiction

Xpredict  = regr.predict(xtest) * (1 + 0.023)

In [461]:
r2_score(ytrain, ypred_train)

0.8860195229695866

In [462]:
r2_score(ytest, ypred_test)

0.4656884447015198

In [463]:
#on un dataframe pour comparer nos prédictions à celle de l'école

FINAL = FM_test[["date","prevision","reel"]]
FINAL["pred"] = Xpredict 
FINAL["marge IA"] = FINAL["pred"] - FINAL["reel"]
FINAL["marge IA"] = FINAL["marge IA"].astype(int)
FINAL["marge ecole"] = FINAL["prevision"] - FINAL["reel"]
# les colonne marge représente la sur ou sous production de repas 

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\3014264685.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINAL["pred"] = Xpredict
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\3014264685.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINAL["marge IA"] = FINAL["pred"] - FINAL["reel"]
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\3014264685.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [464]:
print("le nombre de jours de sous production pour l'école est:", (FINAL["marge ecole"] < 0).sum())
print("le nombre de jours de sous production pour le modèle est:", (FINAL["marge IA"] < 0).sum())

le nombre de jours de sous production pour l'école est: 1
le nombre de jours de sous production pour le modèle est: 6


In [465]:
print("avec ce modele on a reduit le gaspillage alimentaire de",round((FINAL["marge ecole"].sum() - FINAL["marge IA"].sum())/FINAL["marge ecole"].sum()*100, 2)," % par rapport au modele de l'école, en partant du principe qu'il y aura 6 jour avec -200 repas maximum ")

avec ce modele on a reduit le gaspillage alimentaire de 41.34  % par rapport au modele de l'école, en partant du principe qu'il y aura 6 jour avec -200 repas maximum 


In [466]:
Xpredict  = regr.predict(xtest) * (1 + 0.0318)

In [467]:
print("le nombre de jours de sous production pour le modèle est:", (FINAL["marge IA"] < 0).sum())

le nombre de jours de sous production pour le modèle est: 6


In [468]:
FINAL = FM_test[["date","prevision","reel"]]
FINAL["pred"] = Xpredict 
FINAL["marge IA"] = FINAL["pred"] - FINAL["reel"]
FINAL["marge IA"] = FINAL["marge IA"].astype(int)
FINAL["marge ecole"] = FINAL["prevision"] - FINAL["reel"]

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\1222400473.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINAL["pred"] = Xpredict
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\1222400473.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINAL["marge IA"] = FINAL["pred"] - FINAL["reel"]
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\1222400473.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [469]:
print("avec ce modele on a reduit le gaspillage alimentaire de",round((FINAL["marge ecole"].sum() - FINAL["marge IA"].sum())/FINAL["marge ecole"].sum()*100)," % par rapport au modele de l'école, , en partant du principe qu'il y aura O jour de sous productions")

avec ce modele on a reduit le gaspillage alimentaire de 27  % par rapport au modele de l'école, , en partant du principe qu'il y aura O jour de sous productions


# Gradient Boosting Regressor (MEILLEUR)

In [470]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [471]:
# with new parameters
gbr = GradientBoostingRegressor(n_estimators=1000, 
    max_depth=10, 
    learning_rate=0.001, 
    min_samples_split=3)
# with default parameters
gbr = GradientBoostingRegressor()
 
print(gbr)

GradientBoostingRegressor()


In [472]:
gbr.fit(xtrain,ytrain)

C:\Users\Utilisateur\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor()

In [473]:
Xbgr = gbr.predict(xtest) * (1+ 0.044)
ypred_test = gbr.predict(xtest)
r2_score(ytest, ypred_test)

0.3656047554949511

In [474]:
FINALgbr = FM_test[["date","jour","prevision","reel"]]
FINALgbr["pred"] = Xbgr
FINALgbr["marge IA"] = FINALgbr["pred"] - FINALgbr["reel"]
FINALgbr["marge IA"] = FINALgbr["marge IA"].astype(int)
FINALgbr["marge ecole"] = FINALgbr["prevision"] - FINALgbr["reel"]

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\84572754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINALgbr["pred"] = Xbgr
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\84572754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINALgbr["marge IA"] = FINALgbr["pred"] - FINALgbr["reel"]
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\84572754.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [475]:
print("le nombre de jours de sous production pour le modèle est:",(FINALgbr["marge IA"] < 0).sum())
print("le nombre de jours de sous production pour l'ecole est:",(FINALgbr["marge ecole"] < 0).sum())

le nombre de jours de sous production pour le modèle est: 7
le nombre de jours de sous production pour l'ecole est: 1


In [476]:
print("avec ce modele on a reduit le gaspillage alimentaire de",round((FINALgbr["marge ecole"].sum() - FINALgbr["marge IA"].sum())/FINALgbr["marge ecole"].sum()*100)," % par rapport au modele de l'école, , en partant du principe qu'il y aura 6 jour avec -200 repas maximum")

avec ce modele on a reduit le gaspillage alimentaire de 49  % par rapport au modele de l'école, , en partant du principe qu'il y aura 6 jour avec -200 repas maximum


In [477]:
Xbgr = gbr.predict(xtest) * (1+ 0.0595)

In [478]:
print("le nombre de jours de sous production pour le modèle est:",(FINALgbr["marge IA"] < 0).sum())

le nombre de jours de sous production pour le modèle est: 7


In [479]:
FINALgbr = FM_test[["date","jour","prevision","reel"]]
FINALgbr["pred"] = Xbgr
FINALgbr["marge IA"] = FINALgbr["pred"] - FINALgbr["reel"]
FINALgbr["marge IA"] = FINALgbr["marge IA"].astype(int)
FINALgbr["marge ecole"] = FINALgbr["prevision"] - FINALgbr["reel"]

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\84572754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINALgbr["pred"] = Xbgr
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\84572754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FINALgbr["marge IA"] = FINALgbr["pred"] - FINALgbr["reel"]
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_2320\84572754.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [480]:
print("avec ce modele on a reduit le gaspillage alimentaire de",round((FINALgbr["marge ecole"].sum() - FINALgbr["marge IA"].sum())/FINALgbr["marge ecole"].sum()*100)," % par rapport au modele de l'école, , en partant du principe qu'il y aura O jour de sous productions")

avec ce modele on a reduit le gaspillage alimentaire de 24  % par rapport au modele de l'école, , en partant du principe qu'il y aura O jour de sous productions
